# 第3章 類似度に基づく推薦

## 準備

In [1]:
import pprint
import numpy as np

# 上位K件
TOP_K = 3

Du = np.array([
               [5, 3, +1],
               [6, 2, +1],
               [4, 1, +1],
               [8, 5, -1],
               [2, 4, -1],
               [3, 6, -1],
               [7, 6, -1],
               [4, 2, np.nan],
               [5, 1, np.nan],
               [8, 6, np.nan],
               [3, 4, np.nan],
               [4, 7, np.nan],
               [4, 4, np.nan],
])
I = np.arange(Du.shape[0])
x = Du[:,:-1]
ru = Du[:,-1]

Iu = I[~np.isnan(ru)]
Iup = I[ru==+1]
Iun = I[ru==-1]
Iu_not = np.setdiff1d(I, Iu)

## ユーザプロファイル

### 01 好きなアイテム集合に含まれるアイテムの特徴ベクトルの集合

In [2]:
print('x[Iu+] = \n{}'.format(x[Iup]))

x[Iu+] = 
[[5. 3.]
 [6. 2.]
 [4. 1.]]


In [3]:
print('x[Iu+] = \n{}'.format(np.array([x[i] for i in Iup])))

x[Iu+] = 
[[5. 3.]
 [6. 2.]
 [4. 1.]]


### 02 特徴ベクトルの総和

In [4]:
print('sum(x[Iu+]) = {}'.format(np.sum(x[Iup], axis=0)))

sum(x[Iu+]) = [15.  6.]


In [5]:
print('sum(x[Iu+]) = {}'.format(np.sum([x[i] for i in Iup], axis=0)))

sum(x[Iu+]) = [15.  6.]


### 03 ユーザプロファイル

In [6]:
pu = np.array((1 / Iup.size) * np.sum(x[Iup], axis=0))
print('pu = {}'.format(pu))

pu = [5. 2.]


In [7]:
pu = np.array((1 / Iup.size) * np.sum([x[i] for i in Iup], axis=0))
print('pu = {}'.format(pu))

pu = [5. 2.]


## コサイン類似度

### 04 ベクトルの内積
### 05 ユーザプロファイルのノルム
### 06 特徴ベクトルのノルム

In [8]:
def cos(pu, xi):
    """
    コサイン類似度関数：ユーザプロファイルpuとアイテムiの特徴ベクトルxiのコサイン類似度を返す。

    Parameters
    ----------
    pu : ndarray
        ユーザuのユーザプロファイル
    xi : ndarray
        アイテムiの特徴ベクトル

    Returns
    -------
    float
        コサイン類似度
    """
    # 04
    num = pu@xi
#    num = np.dot(pu, xi)
#    d = pu.size
#    num = np.sum([pu[k] * xi[k] for k in range(0, d)])
#    num = np.sum(pu * xi)
    print('num = {}'.format(num))
    # 05
    den_u = np.linalg.norm(pu)
#    den_u = np.sqrt(pu@pu)
#    den_u = np.sqrt(np.sum([pu[k]**2 for k in range(0, d)]))
#    den_u = np.sqrt(np.sum(pu**2))
    print('den_u = {:.3f}'.format(den_u))
    # 06
    den_i = np.linalg.norm(xi)
#    den_i = np.sqrt(xi@xi)
#    den_i = np.sqrt(np.sum([xi[k]**2 for k in range(0, d)]))
#    den_i = np.sqrt(np.sum(xi**2))
#    print('den_i = {:.3f}'.format(den_i))
    print('den_i = {:.3f}'.format(den_i))

    cosine = num / (den_u * den_i)
    return cosine

In [9]:
u = 0
i = 7
print('cos(p{}, x{}) = {:.3f}'.format(u, i, cos(pu, x[i])))
u = 0
i = 11
print('cos(p{}, x{}) = {:.3f}'.format(u, i, cos(pu, x[i])))

num = 24.0
den_u = 5.385
den_i = 4.472
cos(p0, x7) = 0.997
num = 34.0
den_u = 5.385
den_i = 8.062
cos(p0, x11) = 0.783


## 推薦

In [10]:
def score(u, i):
    """
    スコア関数：ユーザuのアイテムiに対するスコアを返す。

    Parameters
    ----------
    u : int
        ユーザuのID（ダミー）
    i : int
        アイテムiのID

    Returns
    -------
    float
        スコア
    """
    return cos(pu, x[i])

### 07 各アイテムに対するスコア
### 08 推薦リスト

In [11]:
def order(u, I):
    """
    順序付け関数：アイテム集合Iにおいて、ユーザu向けの推薦リストを返す。

    Parameters
    ----------
    u : int
        ユーザuのID
    I : ndarray
        アイテム集合

    Returns
    -------
    list
        タプル(アイテムID: スコア)を要素にした推薦リスト
    """
    # 07
#    scores = {}
#    for i in I:
#        scores[i] = score(u, i)
    scores = {i: score(u, i) for i in I}
    print('scores = ')
    pprint.pprint(scores)
    # 08
    rec_list = sorted(scores.items(), key=lambda x:x[1], reverse=True)[:TOP_K]
    return rec_list

In [12]:
u = 0
rec_list = order(u, Iu_not)
print('rec_list = ')
for i, scr in rec_list:
    print('{}: {:.3f}'.format(i, scr))

num = 24.0
den_u = 5.385
den_i = 4.472
num = 27.0
den_u = 5.385
den_i = 5.099
num = 52.0
den_u = 5.385
den_i = 10.000
num = 23.0
den_u = 5.385
den_i = 5.000
num = 34.0
den_u = 5.385
den_i = 8.062
num = 28.0
den_u = 5.385
den_i = 5.657
scores = 
{7: 0.9965457582448796,
 8: 0.9832820049844603,
 9: 0.9656157585206697,
 10: 0.8541985556144386,
 11: 0.783110847498294,
 12: 0.9191450300180578}
rec_list = 
7: 0.997
8: 0.983
9: 0.966
